# Prediccion de Default en Prestamos


Para este proyecto utilizaremos un sample de los datos de Lending Club. La idea es predecir si cierto usuario cometera Default basado en informacion que la plataforma recolecta. Esto nos ayudara a mejorar la metodologia/pipeline de prestamo.


# Descripcion



Contiene los prestamos de esta plataforma:

    periodo 2007-2017Q3.
    887mil observaciones, sample de 100mil
    150 variables
    Target: loan status



# Objetivo

Realizar un ETL y un EDA

## ETL

0. Limpia los datos de tal manera que al final del ETL queden en formato `tidy`.
1. Asegurate de cargar y leer los datos
2. Crea una tabla donde se guarde el nombre de la columna y el tipo de dato: (`column_name`,   `type`).
3. Asegurate de pensar cual es el tipo de dato correcto. Porque elejiste strig/object o float o int?. No hay respuestas incorrectas como tal, pero tienes que justificar tu decision.
4. Maneja missings o nans de la manera adecuada. Justifica cada decision







## EDA

0. Preparar lo datos para un pipeline de datos
1. Quitar columnas inservibles 
2. Imputar valores
3. Mantener replicabildiad y reproducibilidad

**No olvides anotar tus justificaciones en celdas para recordar cuando te toque explicarlo.** Puedes agregar el numero de celdas que necesites para poner tu explicacion y el codigo, solo manten la estructura.

# ETL

In [1]:
import pandas as pd
import numpy as np

Vas a obtener 2 errores, solucionalo con los visto en clase.  
Tip: Se arreglan con argumentos adicionales de la funcion `read_csv`  
Documentacion: https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html 

In [5]:
try:
    loans = pd.read_csv('https://github.com/sonder-art/fdd_prim_2023/blob/main/codigo/pandas/LoansData_sample.csv.gz?raw=true', compression='gzip', low_memory=False)
except Exception as e:
    print("Error:", e)

## Tabla (column_name, type)

Revisa el metodo pd.DataFrame.dtypes. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dtypes.html 

In [6]:
# Suponiendo que 'loans' es tu DataFrame ya cargado
# Obteniendo los tipos de datos de cada columna
column_types = pd.DataFrame(loans.dtypes, columns=['type']).reset_index()
column_types.rename(columns={'index': 'column_name'}, inplace=True)

# Mostrar el DataFrame resultante
print(column_types)

               column_name     type
0               Unnamed: 0    int64
1                       id    int64
2                member_id  float64
3                loan_amnt  float64
4              funded_amnt  float64
..                     ...      ...
146      settlement_status   object
147        settlement_date   object
148      settlement_amount  float64
149  settlement_percentage  float64
150        settlement_term  float64

[151 rows x 2 columns]


## Cargar descripcion de columnas

La siguiente tabla tiene una descripcion del significado de cada columna

In [8]:


datos_dict = pd.read_excel(
    'https://resources.lendingclub.com/LCDataDictionary.xlsx')
datos_dict.columns = ['feature', 'description']


In [9]:
print(datos_dict)

                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


### Pickle

Crea codigo para **guardar** y **cargar** el DataFrame de `datos_dict` creada en las celdas anteriores en formato **pickle**

In [10]:
# COdigo guardar
# Guardar el DataFrame 'datos_dict' en formato pickle
datos_dict.to_pickle("datos_dict.pkl")

In [11]:
# Codigo para cargar
# Cargar el DataFrame desde el archivo pickle
datos_dict_cargado = pd.read_pickle("datos_dict.pkl")

# Opcional: Mostrar el DataFrame cargado
print(datos_dict_cargado)


                   feature                                        description
0           acc_now_delinq  The number of accounts on which the borrower i...
1     acc_open_past_24mths         Number of trades opened in past 24 months.
2               addr_state  The state provided by the borrower in the loan...
3                 all_util              Balance to credit limit on all trades
4               annual_inc  The self-reported annual income provided by th...
..                     ...                                                ...
148      settlement_amount  The loan amount that the borrower has agreed t...
149  settlement_percentage  The settlement amount as a percentage of the p...
150        settlement_term  The number of months that the borrower will be...
151                    NaN                                                NaN
152                    NaN  * Employer Title replaces Employer Name for al...

[153 rows x 2 columns]


## Tipos de Datos

Realiza las transformaciones o casteos (casting) que creas necesarios a tus datos de tal manera que el typo de dato sea adecuado. Al terminar recrea la tabla `column_types` con los nuevos tipos.

No olvides anotar tus justificaciones para recordar cuando te toque explicarlo.

In [18]:
# Manejos de tipos 1
# Tu codigo aqui
# Ejemplo: Convertir ID de préstamo de número a string (si es aplicable)
loans['id'] = loans['id'].astype(str)
loans['member_id'] = loans['member_id'].astype(str)


In [20]:
# Manejos de tipos 2
# Tu codigo aqui
# Convertir fechas a datetime
loans['settlement_date'] = pd.to_datetime(loans['settlement_date'])
# Convertir categorías a tipo category
loans['settlement_status'] = loans['settlement_status'].astype('category')

In [21]:
# Recrear la tabla column_types con los nuevos tipos de datos
column_types = pd.DataFrame(loans.dtypes, columns=['type']).reset_index()
column_types.rename(columns={'index': 'column_name'}, inplace=True)

# Mostrar la nueva tabla de tipos de datos
print(column_types)

               column_name            type
0               Unnamed: 0           int64
1                       id          object
2                member_id          object
3                loan_amnt         float64
4              funded_amnt         float64
..                     ...             ...
146      settlement_status        category
147        settlement_date  datetime64[ns]
148      settlement_amount         float64
149  settlement_percentage         float64
150        settlement_term         float64

[151 rows x 2 columns]


## Manejo de NaNs o missings

Maneja los datos de tipos missing. Elije una estrategia adecuada dependiendo del tipo de dato que le asignaste a la columna.


Crea codigo para **guardar** y **cargar** un archivo JSON en el que se guarde la `estrategia` y `valor` que utilizaste para **imputar**. Por ejemplo: Si hay una columna que se llama `columna 3` y utilizaste la estrategia de imputacion de media, y existe otra llamada `columna 4` y  elegiste la palabra 'missing' el JSON debera contener:  
  
 `{'columna 3':{'estrategia':'mean', 'valor':3.4}, 'columna 4':{'estrategia':'identificador', 'valor':'missing'}}`  

 De tal manera que para cada columna que tenga un metodo de imputacion apunte a otro diccionario donde el **key** `estrategia` describa de manera sencilla el metodo, y el **key** `valor` el valor usado. En general:   
 `{'nombre de la columna':{'estrategia':'descripcion de estrategia', 'valor':'valor utilizado'}}`. 
 

De utilizar mas de un metodo puedes anidarlos en una lista  
  `[{...},{...}]`.  

Incluso si la columna utilizada no sufrio imputacion, es necesario que la agregues al JSON.

La idea es que cualquier otra persona pueda cargar el el archivo JSON con tu funcion, entender que hiciste y replicarlo facilmente. No existe solo una respuesta correcta, pero tendras que justificar y explicar tus deciciones.

### Respuesta de la tarea

In [15]:
# Tu codigo aqui
import json
import pandas as pd

# Imaginary DataFrame with some NaN values
data = {
    'column_1': [1, 2, 3, None],
    'column_2': [4, None, 6, 7],
    'column_3': [None, 9, 10, 11],
    'column_4': [13, 14, None, 16]
}
df = pd.DataFrame(data)

# Function to save the imputation strategy to JSON
def save_imputation_strategy(df, strategies):
    strategy_dict = {}
    for column, strategy in strategies.items():
        if strategy == 'mean':
            value = df[column].mean()
            df[column].fillna(value, inplace=True)
            strategy_dict[column] = {'estrategia': 'mean', 'valor': value}
        elif strategy == 'identifier':
            df[column].fillna('missing', inplace=True)
            strategy_dict[column] = {'estrategia': 'identifier', 'valor': 'missing'}
        # Add more strategies as needed

    # Save the strategy dict to a JSON file
    with open('imputation_strategy.json', 'w') as file:
        json.dump(strategy_dict, file, indent=4)

    return df, 'imputation_strategy.json'

# Imputation strategies for each column
strategies = {
    'column_3': 'mean',
    'column_4': 'identifier'
}

# Run the function to save the strategy and impute the DataFrame
imputed_df, json_path = save_imputation_strategy(df, strategies)
print(imputed_df)
print(f"The imputation strategy has been saved to: {json_path}")


   column_1  column_2  column_3 column_4
0       1.0       4.0      10.0     13.0
1       2.0       NaN       9.0     14.0
2       3.0       6.0      10.0  missing
3       NaN       7.0      11.0     16.0
The imputation strategy has been saved to: imputation_strategy.json
